In [2]:
import onnxruntime as rt
import numpy as np

onnx_model_v1 = "../data/models/birdnet-onnx-v1.onnx"

test = np.random.randn(5, 144_000).astype(np.float32)

sess = rt.InferenceSession(onnx_model_v1)
y = sess.run(None, {"inputs:0": test})
[res.shape for res in y]

[(320,), (3337,)]

In [3]:
import tensorflow as tf

opt_graph_v1 = "../data/models/birdnet-opt-v1.pb"

# load optimized graph
with tf.io.gfile.GFile(opt_graph_v1, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# y = eval_graph(graph_def, test)
with tf.compat.v1.Session() as sess:
    y = tf.graph_util.import_graph_def(
        graph_def,
        input_map={"inputs:0": tf.convert_to_tensor(test)},
        return_elements=["unknown_242:0", "unknown_244:0"],
    )
y

2023-03-19 00:30:36.328908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 00:30:36.437655: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 00:30:36.927112: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-19 00:30:36.927177: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

[<tf.Tensor 'import/unknown_242:0' shape=(320,) dtype=float32>,
 <tf.Tensor 'import/unknown_244:0' shape=(3337,) dtype=float32>]

We're getting the wrong size for the output model, we only get the first element...

In [4]:
tf_bin_path = "../data/bin/linux/"
model_path = "../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"
new_model_path_v2 = "../data/models/birdnet-multi-v2"
frozen_graph_v2 = "../data/models/birdnet-frozen-v2.pb"
opt_graph_v2 = "../data/models/birdnet-opt-v2.pb"
onnx_model_v2 = "../data/models/birdnet-onnx-v2.onnx"

In [5]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)

m = tf.saved_model.load(model_path)

In [16]:
orig = m.model(tf.convert_to_tensor(test)).numpy()
orig

array([[ -9.015548 ,  -8.29387  ,  -7.456644 , ...,  -9.061904 ,
         -8.107562 , -10.288844 ],
       [ -9.267565 ,  -8.835592 ,  -7.3955445, ...,  -9.482044 ,
         -7.9302106, -10.594653 ],
       [ -9.677235 ,  -8.81307  ,  -7.9906316, ...,  -9.728452 ,
         -8.797896 , -10.946492 ],
       [ -9.25017  ,  -8.448324 ,  -7.434323 , ...,  -9.521393 ,
         -8.17159  , -10.56024  ],
       [ -8.8972225,  -8.508541 ,  -7.4888043, ...,  -9.322913 ,
         -8.256334 , -10.647005 ]], dtype=float32)

In [7]:
[x for x in dir(m) if not x.startswith("_")]

['basic',
 'graph_debug_info',
 'model',
 'signatures',
 'tensorflow_git_version',
 'tensorflow_version']

In [8]:
m.basic

In [9]:
m.signatures["basic"]

<ConcreteFunction signature_wrapper(*, inputs) at 0x7FE67C2DF580>

In [11]:
signatures = {
    "basic": m.signatures["basic"],
    "array_input": m.model.__call__.get_concrete_function(
        tf.TensorSpec([None, 144_000], dtype=tf.float32)
    ),
}
print(signatures["basic"].pretty_printed_signature())
print(signatures["array_input"].pretty_printed_signature())

tf.saved_model.save(m, new_model_path_v2, signatures=signatures)

signature_wrapper(*, inputs)
  Args:
    inputs: float32 Tensor, shape=(None, 144000)
  Returns:
    {'scores': <1>}
      <1>: float32 Tensor, shape=(None, 3337)
restored_function_body(inputs, training=False, mask=None)
  Args:
    inputs: float32 Tensor, shape=(None, 144000)
  Returns:
    float32 Tensor, shape=(None, 3337)
INFO:tensorflow:Assets written to: ../data/models/birdnet-multi-v2/assets


INFO:tensorflow:Assets written to: ../data/models/birdnet-multi-v2/assets


In [12]:
# reload new model
m2 = tf.saved_model.load(new_model_path_v2)
m2.signatures

_SignatureMap({'basic': <ConcreteFunction signature_wrapper(*, inputs) at 0x7FE6739BA0B0>, 'array_input': <ConcreteFunction signature_wrapper(*, inputs) at 0x7FE672B45600>})

https://stackoverflow.com/questions/56857043/why-is-convert-variables-to-constants-deprecated-in-tf2

In [149]:
# frozen_func = tf.compat.v1.graph_util.convert_variables_to_constants(m2.signatures["basic"])
# frozen_func = convert_variables_to_constants_v2(m2.basic.get_concrete_function(tf.TensorSpec([None, 144_000],dtype=tf.float32)))
model_func = tf.function(lambda inputs: m2.basic(inputs))
model_func = model_func.get_concrete_function(
    inputs=tf.TensorSpec(None, dtype=tf.float32)
)
frozen_func = convert_variables_to_constants_v2(model_func)

tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph.as_graph_def(),
    logdir="./",
    name=frozen_graph_v2,
    as_text=False,
)

2023-03-19 01:12:19.384587: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-03-19 01:12:19.384704: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


'./../data/models/birdnet-frozen-v2.pb'

In [150]:
res = frozen_func(tf.convert_to_tensor(test))[0].numpy()
np.allclose(orig, res)

True

In [225]:
! {tf_bin_path}/transform_graph \
    --in_graph={frozen_graph_v2} \
    --out_graph={opt_graph_v2} \
    --inputs='inputs:0' \
    --outputs="StatefulPartitionedCall/StatefulPartitionedCall/model/CLASS_DENSE_1/BiasAdd:0,StatefulPartitionedCall/StatefulPartitionedCall/model/GLOBAL_AVG_POOL/Mean:0" \
    --transforms=' \
        remove_nodes(op=Identity, op=CheckNumerics) \
        fold_constants(ignore_errors=true) \
        fold_batch_norms \
        fold_old_batch_norms \
        strip_unused_nodes \
    '
! ls -alh {opt_graph_v2}

2023-03-19 01:39:17.244411: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 01:39:17.292219: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2023-03-19 01:39:17.443771: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying fold_constants
2023-03-19 01:39:17.523662: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying fold_batch_norms
2023-03-19 01:39:17.539693: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying fold_old_batch_norms
2023-03-19 01:39:17.581747: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying strip_unused_nodes
-rw-rw-r-- 1 anthony anthony 22M Mar 19 01:39 ../data/models/birdnet-opt-v2.pb


In [226]:
with tf.io.gfile.GFile(opt_graph_v2, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# y = eval_graph(graph_def, test)
with tf.compat.v1.Session() as sess:
    y = tf.graph_util.import_graph_def(
        graph_def,
        input_map={"inputs:0": tf.convert_to_tensor(test)},
        return_elements=[
            "StatefulPartitionedCall/StatefulPartitionedCall/model/CLASS_DENSE_1/BiasAdd:0",
            "StatefulPartitionedCall/StatefulPartitionedCall/model/GLOBAL_AVG_POOL/Mean:0",
        ],
    )
    pred = y[0].eval()
    emb = y[1].eval()
np.allclose(orig, pred), emb.shape

(True, (5, 320))

In [228]:
! {tf_bin_path}/summarize_graph \
    --in_graph={opt_graph_v2} \
    --print_structure=true

2023-03-19 01:45:03.811664: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Found 1 possible inputs: (name=inputs, type=float(1), shape=<unknown>) 
No variables spotted.
Found 1 possible outputs: (name=StatefulPartitionedCall/StatefulPartitionedCall/model/CLASS_DENSE_1/BiasAdd, op=Add) 
Found 5489531 (5.49M) const parameters, 0 (0) variable parameters, and 345 control_edges
Op types used: 344 Const, 53 Conv2D, 46 Mul, 45 FusedBatchNormV3, 43 Sigmoid, 34 IdentityN, 15 AddV2, 13 Reshape, 13 Pack, 11 StridedSlice, 10 Mean, 10 Shape, 9 DepthwiseConv2dNative, 4 ConcatV2, 3 Relu, 2 Sub, 2 Add, 2 FloorDiv, 2 Pow, 1 Transpose, 1 GatherV2, 1 AvgPool, 1 SplitV, 1 Cast, 1 RealDiv, 1 Range, 1 RFFT, 1 Placeholder, 1 ExpandDims, 1 NoOp, 1 Min, 1 Maximum, 1 MaxPool, 1 Max, 1 MatMul
To u

- https://stackoverflow.com/questions/63618671/how-to-run-inference-using-tensorflow-2-2-pb-file

In [227]:
! python -m tf2onnx.convert \
  --input {opt_graph_v2} \
  --inputs inputs:0[-1,144000] \
  --outputs StatefulPartitionedCall/StatefulPartitionedCall/model/CLASS_DENSE_1/BiasAdd:0,StatefulPartitionedCall/StatefulPartitionedCall/model/GLOBAL_AVG_POOL/Mean:0 \
  --output {onnx_model_v2}

2023-03-19 01:39:49.949788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 01:39:50.055235: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 01:39:50.531913: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-19 01:39:50.531970: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [129]:
with tf.io.gfile.GFile(opt_graph_v2, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# y = eval_graph(graph_def, test)
with tf.compat.v1.Session() as sess:
    y = tf.graph_util.import_graph_def(
        graph_def,
        input_map={"inputs:0": tf.convert_to_tensor(test)},
        return_elements=["unknown_242:0"],
    )
y, test.shape

([<tf.Tensor 'import_31/unknown_242:0' shape=(320,) dtype=float32>],
 (10, 144000))

In [130]:
sess = rt.InferenceSession(onnx_model_v2)
y = sess.run(None, {"inputs:0": test})
[res.shape for res in y]

[(320,), (3337,)]